# Automatic Tests Generation

In [5]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git

In [21]:
import numpy as np
from sklearn.datasets import fetch_openml

from evidently.tests import TestColumnQuantile, TestColumnValueMin, TestColumnShareOfMissingValues
from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests

In [7]:
#datasets generation 
data = fetch_openml(name='adult', version=2, as_frame='auto')

data.frame['target'] = data.frame['education-num']
data.frame['preds'] = data.frame['education-num'].values + np.random.normal(0, 6, data.frame.shape[0])

reference_data = data.frame[:20000]
current_data = data.frame[20000:]

## Multiple tests with list comprehention 

In [8]:
suite = TestSuite(tests=[
    TestColumnQuantile(column_name="education-num", quantile=quantile) for quantile in [0.5, 0.9, 0.99]
])

suite.run(current_data=current_data, reference_data=reference_data)
suite

In [9]:
suite = TestSuite(tests=[
    TestColumnValueMin(column_name=column_name, gt=0) for column_name in ["age", "fnlwgt", "education-num"]
])

suite.run(current_data=current_data, reference_data=reference_data)
suite

## Multiple tests with column test generator

By default, generate_column_tests generates tests with default parameters for all columns in the dataframe

In [22]:
suite = TestSuite(tests=[generate_column_tests(TestColumnShareOfMissingValues)])
suite.run(current_data=current_data, reference_data=reference_data)
suite

### Passing parameters to generate_column_tests

In [23]:
suite = TestSuite(tests=[generate_column_tests(TestColumnShareOfMissingValues, columns="all", parameters={"lt": 0.5})])
suite.run(current_data=current_data, reference_data=reference_data)
suite

### Generate tests for different feature subsets 

In [24]:
suite = TestSuite(tests=[generate_column_tests(TestColumnShareOfMissingValues, columns="all", parameters={"lt": 0.5})])
suite.run(current_data=current_data, reference_data=reference_data)
suite

In [17]:
suite = TestSuite(tests=[generate_column_tests(TestColumnValueMin, columns="num")])
suite.run(current_data=current_data, reference_data=reference_data)
suite

In [25]:
suite = TestSuite(tests=[generate_column_tests(TestColumnShareOfMissingValues, columns="cat", parameters={"lt": 0.1})])
suite.run(current_data=current_data, reference_data=reference_data)
suite

### Generate tests for a custom colums list 

In [19]:
suite = TestSuite(tests=[generate_column_tests(TestColumnValueMin, columns=["age", "fnlwgt", "education-num"], 
                                               parameters={"gt": 0})])
suite.run(current_data=current_data, reference_data=reference_data)
suite